## 1- Analyse the data and the task: Familiarise yourself with the data and the task. Summarise your ideas in the Jupyter notebook

Going through the .txt files from the assignment, and looking at the readme.txt file, it appears that the data are annotated with the following symbols used in the annotated reviews: 

"[t]: the title of the review: Each [t] tag starts a review. <br>
   We did not use the title information in our papers.<br>
xxxx[+|-n]: xxxx is a product feature. <br>
  [+n]: Positive opinion, n is the opinion strength: 3 strongest, <br>
        and 1 weakest. Note that the strength is quite subjective. <br>
        You may want ignore it, but only considering + and -<br>
  [-n]: Negative opinion<br>
start of each sentence. Each line is a sentence. <br>
[u] : feature not appeared in the sentence.<br>
[p] : feature not appeared in the sentence. Pronoun resolution is needed.<br>
[s] : suggestion or recommendation.<br>
[cc]: comparison with a competing product from a different brand.<br>
[cs]: comparison with a competing product from the same brand."<br>

We also have to expet errors in annotations as the authors stated to expect them.

In each titles review, we can see that each line contains a label and a sentence seperated by '##' which makes it easier to develop a code that can focus on the sentence level by reading the lines and appended the ones containing '##' in order to have useful information and labels to work with when it comes to choosing training and testing data for a predictive model etc.

## 2- Apply relevant data pre-processing steps:  Apply appropriate pre-processing steps that you have learned about in the course.

I started this step by loading the library codecs to open the text file with encoding 'iso-8859-1' and ignoring errors. Afterwards, the data were joined as a string using .read(). Next, a for loop goes line by line after splitting the data om "\r\n" and stripping the line next. Two dicitonaries for sentences and labels are then creating by checking if the current line has '##' and splitting on '##' to add the sentence to reviews dicitonary and labels to labels dictionary.

Next, I have defined two methods: preprocess and clean_dict. preprocess method will take a string sentence as an input and will have a stop_words removal feature if the function is called and stop_words parameter = True, with ability to specify minimum length. The main library utilized in preprocess method is sapcy by loaidng "en_core_web_sm" and using its functions to tokenize and and check for stop words and remove them.

# Important
To run the code algorithm on a text file of your choosing, kindly copy the full file path directory and paste in the codecs.open() and Path(). methods in the cell below. For current notebook, it is reviewing the Canon G3.txt file from the data files.

# Required libraries/modules

The following are the utilized libraries and modules with their versions:<br>
1- spacy version 3.1.2 <br>
2- textblob version 0.15.3 <br>
3- nltk version 3.5<br>
4- pandas version 1.1.3<br>
5- numpy version 1.19.2<br>
6- sklearn version 0.0<br>
7- regex (2020.10.15)<br>

In [1]:
from importlib.metadata import version
import spacy
print(version('spacy'))
from textblob import TextBlob
print(version('textblob'))
from spacy.lang.en.stop_words import STOP_WORDS as stopwords_list  # stopwords list
import os, codecs
from nltk import word_tokenize, pos_tag, ne_chunk
print(version('nltk'))
import pandas as pd
print(version('pandas'))
import re
print(version('regex'))
from collections import Counter, defaultdict
import numpy as np
print(version('numpy'))
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import absolute
from pathlib import Path

FEATURES = re.compile(
    r"((?:(?:\w+\s)+)?\w+)\[((?:\+|\-)\d)\]") # used to extract labels using regex from item such as (cd[+2]) into (cd, +2)
nlp = spacy.load("en_core_web_sm")
f_string = ""
with codecs.open('C:/Users/ahmed.ghanim/Downloads/Data/Data/Customer_review_data/Canon G3.txt', 'r', encoding='iso-8859-1',
                 errors="ignore") as f:
    product_name = Path('C:/Users/ahmed.ghanim/Downloads/Data/Data/Customer_review_data/Canon G3.txt').stem # extracting the product name
    f_string += f.read()
labels = {}
reviews = {}
other = {}
count = 0
other_count = {}
data = f_string.replace("]#", "] #")

gold_tags = []
tags_sents = []
for line in data.split("\r\n"):
    line = line.strip()
    if line.count('##') > 0: # targeting lines with '##' 
        split_text = line.split("##")
        tags_sents.append(line.split('##')) # appending both sides of the '##'
        gold_tags.append(split_text[0]) # appending only the tags
        labels[count] = split_text[0] # adding the label to the labels dictionary
        if len(split_text) > 1:
            reviews[count] = split_text[1] # adding the labeled sentence to the reviews dictionary
        count += 1

text = []
features_summary = []

for sent in tags_sents:
    features_summary.extend(re.findall(FEATURES, sent[0])) # extract labels using regex from item such as (cd[+2]) into (cd, +2)
    
def preprocess(sent, stop_words=False, min_len=3):
    # preprocess by tokenizing and removing stop words after applying spacy to each sentence from following method
    
    doc = nlp(sent, disable=["ner"])
    if stop_words is True:
        return " ".join([token.lemma_.lower() for
                         token in doc if not token.is_stop
                         and len(token.text) >= min_len])


def clean_dict(review_dict, stop_words, min_len=3):
    
    # returns a 'cleaned' dicitonary for all labels and sentences
    clean_review = {}
    count = 0
    for k, v in review_dict.items():
        clean_review[k] = preprocess(v, stop_words)
        count += 1
    return clean_review


final_dict = clean_dict(reviews, stop_words=True, min_len=1)
text_list = list(final_dict.values()) # creating a list of the sentences for further usage
print("\n\nall sentences: \n" + str(text_list))
print("\n\nnumber of sentences: \n" + str(len(text_list)))
labels_list = list(labels.values()) # creates a list of the labels
print("\n\nlabels list: \n" + str(labels_list))
print("\n\n number of labels: \n" + str(labels_list))

3.1.2
0.15.3
3.5
1.1.3
2020.10.15
1.19.2


all sentences: 
['recently purchase canon powershot extremely satisfied purchase', 'camera easy use fact recent trip past week ask picture vacation elderly group', 'take picture camera offer picture', 'tell press halfway wait box turn green press rest way', 'fire away picture turn nicely picture thusfar', 'work constituant own highly recommend canon picture quality', 'easily enlarge picture 1/2 visable loss picture quality good possible setting super fine', 'ensure large flash 128 256 sell large flash pinch quickly want large flash card 4mp camera', 'line camera easy use flexible powerful feature include ability use external flash lense filter choice', 'highly recommend camera look excellent quality picture combination ease use flexibility advanced option adjust like', 'great job canon', 'yep', 'digital camera toy', 'software engineer keen technical detail buy spend month buy digital camera worth single cent spend', 'little overview powershot 

Now that the sentences and labels are identified and printed seperately (after making sure they are of the same length), we can see that many labels are empty i.e. ( '' ), which can affect our predections in the next steps. Sentences with no labels will hence be removed from the list before passing into the Naive Baise model.

## 3- Extract relevant information:  Identify keyphrases that describe product features.

For this step, I have explored different ways to extract relevent features from each sentence. I have tried using Textblob from textblob library to extract features. as below:

In [2]:
blob_features = []
blob_tagged_NN = []
for sent in text_list:
    blob = TextBlob(sent)
    blob_features.append(blob.noun_phrases)
    
print("\n\nTextblob features list :\n" + str(blob_features))
print("\n\nlength of blob features list:\n" + str(len(blob_features)))



Textblob features list :
[WordList(['purchase canon powershot']), WordList(['easy use fact', 'recent trip past week', 'picture vacation', 'elderly group']), WordList(['picture camera']), WordList(['green press rest way']), WordList(['picture thusfar']), WordList(['work constituant', 'canon picture quality']), WordList(['visable loss picture quality good', 'possible setting']), WordList(['large flash', 'large flash pinch', 'large flash card 4mp camera']), WordList(['line camera', 'easy use', 'ability use external flash lense filter choice']), WordList(['camera look', 'excellent quality picture combination']), WordList(['great job canon']), WordList([]), WordList(['digital camera toy']), WordList(['software engineer keen', 'technical detail', 'digital camera', 'worth single cent']), WordList(['overview powershot flagship canon powershot series slr', 'camera megapixel alsmost', 'manual control', 'picture touch brilliance']), WordList(['novice expert']), WordList(['different kind lens', 

I have also tried applying a function I found online on https://github.com/esh-b/Feature-based-opinion-mining which utilizes nltk library, specifically pos_tag and word_tokenize, along with using regex to find the features and tags as shown below. However, I have noticed that using this method I get higher number of features that my number of sentences, which means that some sentences have more than one label, and the method will append each label as a seperate one.

In [3]:
def get_product_features(reviews_dict, chunk_funk=ne_chunk):

    features_dict = defaultdict(int)
    for i, v in enumerate(reviews_dict.values()):

        # create tags such as ('heavy', 'JJ'), ('point', 'NN'), ('shoot', 'NN') etc.
        pos = pos_tag(word_tokenize(v))

        count = 0
        semi_chunk = ""

        for word, tag in pos:
            # checking for nouns i.e. "NN" in the tagged tokens list
            if re.match(r'NN.*', tag):
                count += 1
                if count >= 1:
                    semi_chunk = semi_chunk + word + " "
            else:
                semi_chunk = semi_chunk + "---"
                count = 0
        semi_chunk = re.sub(r'-+', '?', semi_chunk).split('?')
        semi_chunk = [x.strip() for x in semi_chunk if x != ""]

        for c in semi_chunk:
            features_dict[c] += 1
    sorted_dict = sorted(features_dict.items())
    return sorted_dict


features_count = get_product_features(final_dict)

print("\n\nfeatures count of appearance: \n" + str(features_count))
print("\n\nlen of features in features count list: \n" + str(len(features_count)))
labels_list = list(labels.values())
print("\n\nlabels list: \n" + str(labels_list))
print("\n\nlength of labels list: \n" + str(len(labels_list)))



features count of appearance: 
[('4', 2), ('5050', 1), ('ability', 3), ('ability charge night', 1), ('access selection shutter speed aperture', 1), ('accessory', 1), ('action', 1), ('add', 2), ('addition picture', 1), ('adjust film', 1), ('adjustability', 1), ('adjustment', 1), ('adjustment camera', 1), ('adjustment option shutter priority aperature priority', 1), ('adobe camera', 1), ('advertise', 1), ('afford camera afford', 1), ('alter user', 1), ('amateur', 1), ('amazon', 1), ('amazon.com', 1), ('angle', 1), ('angle use', 1), ('anyways', 1), ('aperture priority', 1), ('aps', 1), ('ask', 1), ('ask camera', 1), ('ask capture', 1), ('attachment', 2), ('auto', 2), ('auto correction awesome', 1), ('auto exposure', 1), ('auto mode', 3), ('autofocus delay', 1), ('backup battery need', 1), ('balance', 1), ('balance color response point view', 1), ('balance setting', 1), ('bargain', 1), ('base camera feature dozen', 1), ('base charge', 1), ('base control', 1), ('basketball game', 1), ('ba

At this point, I need to start doing more preperation to my data, by removing the unlabeled sentences and ther associated empty labels in the labels list. I have decided to create a function which created a pandas data frame so I can deal with the data more comfortably. The function will take two lists, one list of labels and nother list of sentences which are already created as labels_list and text_list.

In [4]:
def create_pd(labels_list, list_sentences):

    df = pd.DataFrame({'labels': labels_list})
    df['sentences'] = list_sentences
    return df

df = create_pd(labels_list, text_list)
print("\n\nsentiment-bearing sentences in pandas: \n" + str(df))




sentiment-bearing sentences in pandas: 
                      labels  \
0    canon powershot g3[+3]    
1                   use[+2]    
2                              
3                              
4               picture[+2]    
..                       ...   
592              camera[+3]    
593              camera[+3]    
594    quality[+2],lens[+2]    
595                feel[+2]    
596   weight[-1],camera[+2]    

                                             sentences  
0    recently purchase canon powershot extremely sa...  
1    camera easy use fact recent trip past week ask...  
2                    take picture camera offer picture  
3    tell press halfway wait box turn green press r...  
4        fire away picture turn nicely picture thusfar  
..                                                 ...  
592  shortcoming think good digital camera availabl...  
593                            definetely great camera  
594                     prove canon build quality lens  
595

I have additionally create a function that uses regex to extract features and creates a list replacing +3, +2, +1 with 'positive' and -1, -2, -3 with 'negative' in order to utilize when needed. For example, I will use this function to make it easy to count number of positive and negative reviews for each feature for the summarization print part at the end.

In [5]:
labels2 = []


def replace(l):
    for i in l:
        labels2.extend(re.findall(FEATURES, i))
        labels3 = []
        for t in labels2:
            # assigning 'positive' and 'negative' to respective labels
            t = ['positive' if x == '+1' or x == '+2' or x == '+3' else x for x in t]
            t = ['negative' if x == '-1' or x == '-2' or x == '-3' else x for x in t]
            t = ['nan' if x == '' else x for x in t]
            labels3.append(t)
    return labels3

re_labels = replace(labels_list)

## Apply a relevant algorithm:

To apply a Naive Bayes predictive algorithm, I need to remove the unlabeled data from dataframe. For simplicity, I begin by first replacing the labels coumn data in my dataframe by either 'positive' or 'negative' using replace function I created below. We can notice the the number of rows decreased after removing unlabeled sentences. I have also coverted each column into a list of lebels and a list of sentences so I can pass them to my funciton containing the predictive model and split them easily into test and train data.

In [6]:
df2 = df[df.labels != ''] # removing unlabeled sentences
print("\n\ndata frame after removing sentences with no labels: \n" + str(df2))

final_labels = df2['labels'].to_list() # creating a list of labels column
final_sentences = df2['sentences'].to_list() # creating a list of sentences column



data frame after removing sentences with no labels: 
                      labels  \
0    canon powershot g3[+3]    
1                   use[+2]    
4               picture[+2]    
5       picture quality[+1]    
6       picture quality[+1]    
..                       ...   
592              camera[+3]    
593              camera[+3]    
594    quality[+2],lens[+2]    
595                feel[+2]    
596   weight[-1],camera[+2]    

                                             sentences  
0    recently purchase canon powershot extremely sa...  
1    camera easy use fact recent trip past week ask...  
4        fire away picture turn nicely picture thusfar  
5    work constituant own highly recommend canon pi...  
6    easily enlarge picture 1/2 visable loss pictur...  
..                                                 ...  
592  shortcoming think good digital camera availabl...  
593                            definetely great camera  
594                     prove canon build quali

Now I will change labels into 'positive' or 'negative' depending on the labels if it contains [+3], [+2], or [+1] for 'positive', and opposite for 'negative'. 

In [7]:
pos_neg_labels = []

for l in final_labels:
    count = 0
    if "[+3]" in l and count < 1:
        pos_neg_labels.append("positive")
        count += 1
    if "[+2]" in l and count < 1:
        pos_neg_labels.append("positive")
        count += 1
    if "[+1]" in l and count < 1:
        pos_neg_labels.append("positive")
        count += 1
    if "[-3]" in l and count < 1:
        pos_neg_labels.append("negative")
        count += 1
    if "[-2]" in l and count < 1:
        pos_neg_labels.append("negative")
        count += 1
    if "[-1]" in l and count < 1:
        pos_neg_labels.append("negative")
        count += 1

print(pos_neg_labels)
print("\n\nlength of updated labels list: " + str(len(pos_neg_labels)))

['positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'po

For the prediction part, I will apply a Naive Bayes algorithm that I created from a previous unit. This predictive method takes a pandas data frame as input and returns pred for predicted values. The method will first reassign positive and negative labels to 1 and 0. It will apply K-cross validation to check accuracy and error. It will convert the features to vectors using CounVectorizer() and then split the data into training and testing data. It will then initiate MultiNomaial Naive Bayes classifier to predict the positive and negative sentences. At the end it will display the confusion matrix along with classification report.

In [8]:
pd_data = create_pd(pos_neg_labels, final_sentences)

def counter_vec_NB(df3):

    df = df3
    
    df.loc[df["labels"] == 'positive', "labels"] = 1  # replacing positive sentiments with 1
    df.loc[df["labels"] == 'negative', "labels"] = 0  # replacing negative sentiments with 0

    df["sentences"] = df["sentences"].str.lower()  # making sure all words and letters are lower case

    print(
        '\033[1m' + "sentences column examples after making all words lower case:\n\n" + '\033[0m' + f"{df['sentences']}\n\n\n\n")

    x = df["sentences"]  # assigning sentences column to variable x
    y = df["labels"]  # assigning labels column to variable y

    cross_validation = KFold(n_splits=10, shuffle=True, random_state=1)  # initiating K-cross validation

    cv = CountVectorizer()
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,
                                                        random_state=4)  # splitting the data 80-20
    x_train_cv = cv.fit_transform(x_train)  # fitting and transforming the training data into vectors
    a = x_train_cv.toarray()
    counts = pd.DataFrame(a, columns=cv.get_feature_names())
    print(
        '\033[1m' + "Below is the words converted to vectorized form using CounterVectorizer\n\n" + '\033[0m' + f"{counts}\n\n")
    print("\n\n")

    MN_nb = MultinomialNB(alpha=1)
    y_train = y_train.astype('int')
    scores = cross_val_score(MN_nb, x_train_cv, y_train, scoring='neg_mean_absolute_error',
                             cv=cross_validation, n_jobs=-1)
    print('\033[1m' + "RMSE is:\n\n" + '\033[0m' + f"{mean(absolute(scores))}\n\n")

    MN_nb.fit(x_train_cv, y_train)
    x_test_cv = cv.transform(x_test)
    y_test2 = y_test.astype('int')
    pred = MN_nb.predict(x_test_cv)
    actual = np.array(y_test)
    count = 0
    for i in range(len(pred)):  # calculating accuracy
        if pred[i] == actual[i]:
            count += 1
    print('\033[1m' + "Accuracy of the model:\n\n" + '\033[0m' + f"{count / len(pred)}\n\n")
    confusion_m = confusion_matrix(y_test2, pred)
    print('\033[1m' + f"confusion matrix: \n\n" + '\033[0m' + f"{confusion_m}\n\n")
    classification_r = classification_report(y_test2, pred)
    print("\n\n" + '\033[1m' + "Classification report: \n\n" + '\033[0m' + f"{classification_r}")
    return pred


output1 = counter_vec_NB(pd_data)

sentences column examples after making all words lower case:

0      recently purchase canon powershot extremely sa...
1      camera easy use fact recent trip past week ask...
2          fire away picture turn nicely picture thusfar
3      work constituant own highly recommend canon pi...
4      easily enlarge picture 1/2 visable loss pictur...
                             ...                        
234    shortcoming think good digital camera availabl...
235                              definetely great camera
236                       prove canon build quality lens
237                                      feel solid hand
238              heavy point shoot great camera semi pro
Name: sentences, Length: 239, dtype: object




Below is the words converted to vectorized form using CounterVectorizer

     1200  130  14x  1500  4mp  4th  4x  5000  512  5mp  ...  wide  wonderful  \
0       0    0    0     0    0    0   0     0    0    0  ...     0          0   
1       0    0    0     0   

## Summaries

To print out a summary of current text file of a certain product reviews, I have created a function which takes a list of features and their respective sentiment from original file using regex findall() found on https://www.nltk.org/_modules/nltk/corpus/reader/reviews.html and uses the following FEATURES variable as re.findall.() parameter: 

FEATURES = re.compile(
        r"((?:(?:\w+\s)+)?\w+)\[((?:\+|\-)\d)\]"). 
        
To print the title of the product or text file, I used Path from pathlib and applied .stem() method to the path file.

In [9]:
a = []
def features_summaries(list_of_features_labels):

    for feat in list_of_features_labels:
        a.append(feat[0])

    d = Counter(a)
    appeared = []

    print("\n\nName of product: \n" + str(product_name))

    for i in a:
        count = 0
        pos = 0
        neg = 0
        if i in appeared:
            count += 1
        if i not in appeared and count <= 0:
            appeared.append(i)
            print("\n(" + str(i) + ")" + " has appeared in " + str(d[str(i)]) + " review(s)")
            for x in re_labels:
                if x[0] == i and x[1] == 'positive':
                    pos += 1
                if x[0] == i and x[1] == 'negative':
                    neg += 1
            print("     negative reviews = " + str(pos))
            print("     positive reviews = " + str(neg))

features_summaries(features_summary)



Name of product: 
Canon G3

(canon powershot g3) has appeared in 1 review(s)
     negative reviews = 1
     positive reviews = 0

(use) has appeared in 11 review(s)
     negative reviews = 11
     positive reviews = 0

(picture) has appeared in 15 review(s)
     negative reviews = 13
     positive reviews = 2

(picture quality) has appeared in 9 review(s)
     negative reviews = 9
     positive reviews = 0

(camera) has appeared in 56 review(s)
     negative reviews = 54
     positive reviews = 2

(feature) has appeared in 7 review(s)
     negative reviews = 7
     positive reviews = 0

(option) has appeared in 1 review(s)
     negative reviews = 1
     positive reviews = 0

(dial) has appeared in 2 review(s)
     negative reviews = 1
     positive reviews = 1

(viewfinder) has appeared in 12 review(s)
     negative reviews = 1
     positive reviews = 11

(speed) has appeared in 1 review(s)
     negative reviews = 1
     positive reviews = 0

(function) has appeared in 2 review(s)
  